# TensorFlow roBERTa Starter - LB 0.705
This notebook is a TensorFlow template for solving Kaggle's Tweet Sentiment Extraction competition as a question and answer roBERTa formulation. In this notebook, we show how to tokenize the data, create question answer targets, and how to build a custom question answer head for roBERTa in TensorFlow. Note that HuggingFace transformers don't have a `TFRobertaForQuestionAnswering` so we must make our own from `TFRobertaModel`.

# Load Libraries, Data, Tokenizer
We will use HuggingFace transformers [here][1]

[1]: https://huggingface.co/transformers/

In [1]:
pip install transformers

     |████████████████████████████████| 4.0 MB 27.3 MB/s 
     |████████████████████████████████| 596 kB 59.9 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 
     |████████████████████████████████| 6.6 MB 64.6 MB/s 
     |████████████████████████████████| 895 kB 58.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I hate you'))"

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Downloading: 100% 629/629 [00:00<00:00, 463kB/s]
Downloading: 100% 255M/255M [00:15<00:00, 17.7MB/s]
Downloading: 100% 48.0/48.0 [00:00<00:00, 39.1kB/s]
Downloading: 100% 226k/226k [00:00<00:00, 269kB/s]
[{'label': 'NEGATIVE', 'score': 0.9991129040718079}]


In [5]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
#from transformers import *
import tokenizers
print('TF version',tf.__version__)

TF version 2.8.0


In [6]:

from google.colab import files
files.upload()

Saving train.csv to train (1).csv


{'train.csv': b'textID,text,selected_text,sentiment\ncb774db0d1," I`d have responded, if I were going","I`d have responded, if I were going",neutral\n549e992a42, Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative\n088c60f138,my boss is bullying me...,bullying me,negative\n9642c003ef, what interview! leave me alone,leave me alone,negative\n358bd9e861," Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",negative\n28b57f3990,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral\n6e0c6d75b1,2am feedings for the baby are fun when he is all smiles and coos,fun,positive\n50e14c0bb8,Soooo high,Soooo high,neutral\ne050245fbd, Both of you,Both of you,neutral\nfc2cbefa9d, Journey!? Wow... u just became cooler.  hehe... (is that possible!?),Wow... u just became cooler.,positive\n2339a9b08b," as much as i lo

In [8]:
!unzip -q "../content/tweet-sentiment-extraction.zip"

unzip:  cannot find or open ../content/tweet-sentiment-extraction.zip, ../content/tweet-sentiment-extraction.zip.zip or ../content/tweet-sentiment-extraction.zip.ZIP.


In [9]:
 from transformers import RobertaTokenizer

In [10]:
MAX_LEN = 96

tokenizer = RobertaTokenizer.from_pretrained('roberta-base') # https://huggingface.co/roberta-base

sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('../content/train.csv').fillna('')
train.head()

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment  
0  I`d have responded, if I were going   neutral  
1                             Sooo SAD  negative  
2                          bullying me  negative  
3                       leave me alone  negative  
4                        Sons of ****,  negative

# Training Data
We will now convert the training data into arrays that roBERTa understands. 

In [11]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc:#.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc)+5] = [0] + enc + [2,2] + [s_tok] + [2]
    attention_mask[k,:len(enc)+5] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

# Test Data
We must tokenize the test data exactly the same as we tokenize the training data

In [12]:
test = pd.read_csv('../content/test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc)+5] = [0] + enc + [2,2] + [s_tok] + [2]
    attention_mask_t[k,:len(enc)+5] = 1

# Build roBERTa Model
We use a pretrained roBERTa base model and add a custom question answer head. First tokens are input into `bert_model` and we use BERT's first output, i.e. `x[0]` below. These are embeddings of all input tokens and have shape `(batch_size, MAX_LEN, 768)`. Next we apply `tf.keras.layers.Conv1D(filters=1, kernel_size=1)` and transform the embeddings into shape `(batch_size, MAX_LEN, 1)`. We then flatten this and apply `softmax`, so our final output from `x1` has shape `(batch_size, MAX_LEN)`. These are one hot encodings of the start tokens indicies (for `selected_text`). And `x2` are the end tokens indicies.

![bert.jpg](attachment:bert.jpg)

In [13]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    #config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    #bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    bert_model = TFRobertaModel.from_pretrained('roberta-base')
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Conv1D(1,1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(1,1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

# Metric

In [14]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [15]:
from transformers import TFRobertaModel

In [16]:
import tensorflow as tf
import numpy as np

# Train roBERTa Model
We train with 5 Stratified KFolds (based on sentiment stratification). Each fold, the best model weights are saved and then reloaded before oof prediction and test prediction. Therefore you can run this code offline and upload your 5 fold models to a private Kaggle dataset. Then run this notebook and comment out the line `model.fit()`. Instead your notebook will load your model weights from offline training in the line `model.load_weights()`. Update this to have the correct path. Also make sure you change the KFold seed below to match your offline training. Then this notebook will proceed to use your offline models to predict oof and predict test.

In [17]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=2,shuffle=True,random_state=777)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
        
    sv = tf.keras.callbacks.ModelCheckpoint(
        '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')
        
    model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
        epochs=3, batch_size=32, verbose=DISPLAY, callbacks=[sv],
        validation_data=([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], 
        [start_tokens[idxV,], end_tokens[idxV,]]))
    
    print('Loading model...')
    model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    
    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc[a-1:b])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()

#########################
### FOLD 1
#########################


Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
430/430 [==============================] - ETA: 0s - loss: 2.3553 - activation_loss: 1.2023 - activation_1_loss: 1.1530
Epoch 1: val_loss improved from inf to 1.70845, saving model to v0-roberta-0.h5
430/430 [==============================] - 374s 818ms/step - loss: 2.3553 - activation_loss: 1.2023 - activation_1_loss: 1.1530 - val_loss: 1.7085 - val_activation_loss: 0.8438 - val_activation_1_loss: 0.8647
Epoch 2/3
430/430 [==============================] - ETA: 0s - loss: 1.6002 - activation_loss: 0.8011 - activation_1_loss: 0.7991
Epoch 2: val_loss improved from 1.70845 to 1.59871, saving model to v0-roberta-0.h5
430/430 [==============================] - 352s 820ms/step - loss: 1.6002 - activation_loss: 0.8011 - activation_1_loss: 0.7991 - val_loss: 1.5987 - val_activation_loss: 0.8028 - val_activation_1_loss: 0.7959
Epoch 3/3
430/430 [==============================] - ETA: 0s - loss: 1.3575 - activation_loss: 0.6882 - activation_1_loss: 0.6693
Epoch 3: val_loss did not im

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
430/430 [==============================] - ETA: 0s - loss: 2.3818 - activation_loss: 1.2133 - activation_1_loss: 1.1685
Epoch 1: val_loss improved from inf to 1.69553, saving model to v0-roberta-1.h5
430/430 [==============================] - 371s 827ms/step - loss: 2.3818 - activation_loss: 1.2133 - activation_1_loss: 1.1685 - val_loss: 1.6955 - val_activation_loss: 0.8741 - val_activation_1_loss: 0.8215
Epoch 2/3
430/430 [==============================] - ETA: 0s - loss: 1.6636 - activation_loss: 0.8261 - activation_1_loss: 0.8375
Epoch 2: val_loss improved from 1.69553 to 1.57889, saving model to v0-roberta-1.h5
430/430 [==============================] - 352s 819ms/step - loss: 1.6636 - activation_loss: 0.8261 - activation_1_loss: 0.8375 - val_loss: 1.5789 - val_activation_loss: 0.7997 - val_activation_1_loss: 0.7792
Epoch 3/3
430/430 [==============================] - ETA: 0s - loss: 1.3999 - activation_loss: 0.7040 - activation_1_loss: 0.6958
Epoch 3: val_loss did not im

In [18]:
print('>>>> OVERALL 5Fold CV Jaccard =',np.mean(jac))

>>>> OVERALL 5Fold CV Jaccard = 0.5030409254618561


# Kaggle Submission

In [19]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc[a-1:b])
    all.append(st)

In [20]:
test['selected_text'] = all
test[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test.sample(25)

textID                                                         text  \
2840  e2d8e05859    I`ve run out of earl grey, so having lemon tea instead...   
3214  2280788f99                                               crash in Qmbol   
437   d17a3a37e1  Kelly Kelly pays your bills @ 12:15, Kaden pays em @ 5:1...   
83    a14b4ff440  Happy Birthday Snickers!!!! ? I hope you have the best d...   
3521  2b31cedc6d                 munchin BACON BUTTIES! woohoo!!  my faves!!!   
2224  71617c28b6                                Amazing night at the drive in   
3080  fe51ca37d4         guess I missed out on more than beer and a good time   
2483  95dbb10a09  Driving home to trade cars  hopefully it makes it! http:...   
1484  8ba9d3448f                               #warmfuzzies to you, my friend   
1539  0f9b915617                                                 Sunburn ouch   
1962  2a2c1635c8  639 baby! GOD ITS SAD REALLY  oh well lifes a partaaay! ...   
3135  e33459f8f1  Just left Brooklyn going uptown for some latin coffee wi...   
2210  cad8db609a              Why am I having such a hard time falling asleep   
489   5a42a51462      feels ..my cousin is a monkey  http://plurk.com/p/x23pa   
2337  5055018a65                            when u go, LMK. Let`s go together   
1797  ed539778c1  to be really really... not bored but bad, i won`t know t...   
189   f5d47f04f5   Too much? No, not unless one of the pillows ends up suf...   
2090  4363c63d9c          WOOHOOO very cool see I knew u would get to see her   
1818  5be8565b8d                            Im sad i cant go. it makes me sad   
2779  d9bc353713                           farewelling an employee tonight...   
2720  e5d88bca69                                                    jia you !   
3235  665dcd4cf2   Yes.  Me too. I can`t understand why they have to do th...   
589   7315faa6ce                        oh mannn i`m gonna be there tomorroww   
2003  81de3c6143  The day started so wonderful, but now our kids our cryin...   
2928  e540ebed83   haha! i see...  go, wear BLUE! ) heey, ur watching JONA...   

     sentiment                                                selected_text  
2840  negative                                <s> I`ve run out of earl grey  
3214  negative                                                    <s> crash  
437    neutral  <s> Kelly Kelly pays your bills @ 12:15, Kaden pays em @...  
83    positive                                                    <s> Happy  
3521  positive                                         IES! woohoo!! my fav  
2224  positive                                                  <s> Amazing  
3080  negative                                               guess I missed  
2483  positive                                               cars hopefully  
1484  positive                           <s> #warmfuzzies to you, my friend  
1539  negative                                                <s> Sunburn o  
1962   neutral  <s> 639 baby! GOD ITS SAD REALLY oh well lifes a partaaa...  
3135   neutral  <s> Just left Brooklyn going uptown for some latin coffe...  
2210  negative                                                  such a hard  
489   negative                             <s> feels..my cousin is a monkey  
2337  positive                                        MK. Let`s go together  
1797   neutral  <s> to be really really... not bored but bad, i won`t kn...  
189   negative  <s> Too much? No, not unless one of the pillows ends up ...  
2090  positive                                                OOO very cool  
1818  negative                        <s> Im sad i cant go. it makes me sad  
2779   neutral                          <s> farewelling an employee tonight  
2720   neutral                                                  <s> jia you  
3235  positive                                                  called love  
589    neutral                     <s> oh mannn i`m gonna be there tomorrow  
2003   neutral  <s> The day sta

In [21]:
test[['textID','selected_text']].sample(25)

textID                                                selected_text
788   9d4dfb1f9e                                     <s> is downloading songs
1633  a0e8168015    <s> Going to twitter in english, for my foreign followers
1111  80a1ed08bf                                                      <s> Sad
1965  19f108c62c  <s> Hey don`t write yourself off yet. Little girl you`re...
355   d0c092c3ee                                                    <s> Sadly
3251  e37a7b9128       <s> Ouch, give me a heads up so I`ll know when to duck
1889  9114d80bac                              <s> Laundry time and LOTS of it
1037  43d10bd312                                           really frickin hot
1248  fd9f6c3e21                                          morning. That sucks
2885  c7e52408c8                                                  left, bored
1098  44da06d16c               <s> i really wanna go to australia and england
1373  bd3b30d13c                                      hey, happy mother`s day
1429  c9d4cf96af                                       yesterday shortchanged
565   8d8c915ea5                                                 <s> _eat_out
1211  1137d77ecc      <s>....got your message!!! You are such a twitter freak
1462  d6738c3768                                          <s> Excellent point
2035  389fc0d397                         <s> Last scheduled corn dog oc shift
2849  34e4bf2b94   Hi! Waking up, and not lazy at all. You would be proud ...
691   08f6036add  <s> Invasion of the Old Ladies has just ended, still hav...
1029  88a120f336                      <s> Happy mothers day mom!!! i love you
1303  3da51fc31b                                   <s> Inorite, humid as well
317   a2bbae6c4c                                                   access fun
617   7e46f3f92a  <s> ha. nothing any man does is right on mothers day exc...
1515  a5abb95859                                                    ahh Happy
1030  52fc580b72                               ,,,,,,,,, **** outa blank disc